In [1]:
import os
from qgis.core import (
    QgsVectorLayer,
    QgsProject,
    QgsExpression,
    QgsExpressionContext,
    QgsFeatureRequest,
    QgsGeometry,
    QgsFeature,
    
)


In [2]:
os.getcwd()

'c:\\Users\\milan\\OneDrive - MUNI\\VŠ\\PhD\\Zahraniční stáž\\Work\\HeiGIT_notebooks\\analysis\\ai-assisted-osm-mapping-stats\\notebooks-data-prep'

In [3]:
# Set the project home directory
os.chdir("..")
home_wd = os.getcwd()

In [4]:
state_dic = {
    0: "India",
    1: "Vietnam",
    2: "Tanzania",
    3: "Kenya",
    4: "Nigeria",
}

In [5]:
while True:
    try:
        user_data_spec = int(input("Which grid data do you want to split?\
                            Enter a number:\
                            0 for India,\
                            1 for Vietnam,\
                            2 for Tanzania,\
                            3 for Kenya,\
                            4 for Nigeria,\
                                :"
                                ))
    except ValueError:
        print("Please enter a number.")
        continue
    if user_data_spec not in range(len(state_dic)):
        print(f"Please enter one of these numbers: {state_dic.keys()}.")
        continue
    break

In [6]:
# Define the input and output files
in_grid = os.path.join(home_wd, "shp", "origin_downloaded_layers", "heigit_hexagrid.shp")
in_states = os.path.join(home_wd, "shp", "origin_downloaded_layers", "ne_10m_admin_0_sovereignty.shp")
out_grid_geojson = os.path.join(home_wd, "geojson-hexagrid-states", state_dic[user_data_spec])


In [8]:
project = QgsProject.instance()

In [9]:
# Create a QGIS vector layer from the input grid file
grid_layer = QgsVectorLayer(in_grid, "Grid", "ogr")
if not grid_layer.isValid():
    print("Grid layer not valid!")
    exit()

In [10]:
# Create a QGIS vector layer from the input states file
state_layer = QgsVectorLayer(in_states, "States", "ogr")
if not state_layer.isValid():
    print("State layer not valid!")
    exit()

In [11]:
project.addMapLayer(grid_layer)
project.addMapLayer(state_layer)

<QgsVectorLayer: 'States' (ogr)>

In [12]:
# Define the state to be processed
state_name = state_dic[user_data_spec]

In [13]:
# Create an expression to select the desired state
# exp_context = QgsExpressionContext()
# exp_context.appendScopes(QgsExpressionContextUtils.globalProjectLayerScopes(state_layer))
# exp_context.appendScopes(QgsExpressionContextUtils.projectScopes())
# exp_context.setVariable("state_name", state_name)
state_exp = QgsExpression('NAME_EN = \'' + state_dic[user_data_spec] + '\'')
# state_exp.prepare(exp_context)

In [14]:
state_exp.expression()

"NAME_EN = 'Tanzania'"

In [15]:
state_layer.selectByExpression(state_exp.expression(), QgsVectorLayer.SetSelection)

In [16]:
state_layer.selectedFeatureCount()

1

In [16]:
# Create an empty list to store the selected grid features
selected_features = []

In [18]:
# Iterate through the selected state features and select the corresponding grid features
for state_feat in state_layer.selectedFeatures():
    state_geom = state_feat.geometry()
    grid_request = QgsFeatureRequest().setFilterRect(state_geom.boundingBox())
    for grid_feat in grid_layer.getFeatures(grid_request):
        if grid_feat.geometry().intersects(state_geom):
            selected_features.append(grid_feat)

In [17]:
state_geom = state_layer.geometry()
grid_request = QgsFeatureRequest().setFilterRect(state_geom.boundingBox())
for grid_feat in grid_layer.getFeatures(grid_request):
    if grid_feat.geometry().intersects(state_geom):
        selected_features.append(grid_feat)

AttributeError: 'QgsVectorLayer' object has no attribute 'geometry'

In [19]:
state_bbox = state_layer.extent()
grid_request = QgsFeatureRequest().setFilterRect(state_bbox)
for grid_feat in grid_layer.getFeatures(grid_request):
    if grid_feat.geometry().intersects(state_bbox):
        selected_features.append(grid_feat)

In [20]:
selected_features

[]

In [21]:
# Iterate through the selected grid features and export them to GeoJSON files
for grid_feat in selected_features:
    grid_geom = grid_feat.geometry()
    grid_id = grid_feat["id_int"]
    out_file = os.path.join(out_grid_geojson, f"{grid_id}_{state_name}.geojson")
    with open(out_file, "w") as f:
        f.write(grid_geom.asJson())